In [1]:
from sympy import *
from sympy.combinatorics.partitions import IntegerPartition
import numpy as np
from itertools import *

# https://docs.sympy.org/latest/modules/polys/reference.html
# https://github.com/stla/jackpy/tree/main

# Partitions*

In [2]:
# Given the bosonic state k, we can write the bosonic partition as 
# defined in decreasing order
def bosonicPart(k):
    '''This snippet converts the characteristic class notation to 
    the Young diagram notation.'''
    partition = np.array([])
    for i in range(len(k)):
        row = list(np.repeat(i+1, k[i]))
        partition = np.concatenate((partition, row), axis=0)
    partition = -np.sort(-partition, axis=0) # This line sorts the numpy array in decreasing order. 
    return list(partition)

def number_boxes(k):
    box = 0 
    for j in range(len(k)):
        box += (j+1)*k[j]
    if box != int(sum(bosonicPart(k))): 
        # Here I test to see if everything is consistent
        return print("This method does not work!")
    return box

def number_rows(k):
    rows = sum(k)
    if rows != int(len(bosonicPart(k))): 
        # Here I test to see if everything is consistent
        return print("This method does not work!")
    return rows

def number_columns(k):
    return int(bosonicPart(k)[0])
    

def transpose_young_diagram(partition):
    '''Here we find the transposed Young diagram. 
    In order to avoind confusion, here we enter 
    the Young diagram as a partition of the integer
    I explain the code with the partition [3,2].'''
        
    # Number of rows in the original diagram
    num_rows = len(partition)

    '''Here we have a List to represent the transposed diagram. 
    It must have partition[0] rows. 
    Then, for [3,2] we create the list [0,0,0]'''
    #transposed_diagram = [0] * max(partition)
    transposed_diagram = [0] * partition[0]

    '''We now build the transposed diagram. We basically fill in 
    the list defined above. For the list [0,0,0]. The code works as follows:
    round 01: (i, row_length) = (0,3) => j =0,1,2. 
               j = 0 transporsed_diagram[0] = 1
               j = 1 transposed_diagram[1] = 1
               j = 2 transposed diagram[2] = 1
    and now we have transposed_diagram = [1,1,1]
    round 02 (i, row_length) = (1, 2) => j =0,1
               j = 0 transposed_diagram[0] = 2
               j = 1 transposed_diagram[0] = 2
    and now we have transposed_diagram = [2,2,1]'''
    for i, row_length in enumerate(partition):
        for j in range(row_length):
            transposed_diagram[j] += 1

    return transposed_diagram

def transpose_young_diagram_SYMPY(partition):
    '''Sympy has this implemented as well. It might be 
    interesting to use some of these definitions.''' 
    a = IntegerPartition(partition)
    return a.conjugate

def number_diagonal(partition): 
    '''In order to calculate the number of boxes in the 
    diagonal, I will represent the Young diagram as a 
    partition[0] x len(partition) matrix filled with 1 and 0. 
    After that, I sum over the diagonal.'''
    rows = len(partition)
    columns = partition[0]
    matrix = np.zeros((rows, columns))
    diagonal = 0
    for row in range(rows):
        for column in range(partition[row]):
            matrix[row][column] = 1
    return int(np.trace(matrix))

def frobenius_coordinates(partition):
    '''Now I want to find the Frobenius Coordinates 
    for a given partition.'''
    transpose = transpose_young_diagram(partition)
    diagonal_boxes = number_diagonal(partition)
    FrobCoor = []
    for i in range(diagonal_boxes):
        # The minus comes from the fact that python lists start at 0 and not 1
        FrobCoor.append([partition[i] - i - Rational(1,2), -(transpose[i] - i - Rational(1,2))])
    return FrobCoor

# Complete Symmetric Homogeneous Polynomials*

In [5]:
# Miwa coordinates
t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11, t12 = symbols('t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11, t12')
t = np.array([t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11, t12])

# Vector k
k1, k2, k3, k4, k5, k6, k7, k8, k9, k10, k11, k12 = symbols('k1, k2, k3, k4, k5, k6, k7, k8, k9, k10, k11, k12')
k_vec = np.array([k1, k2, k3, k4, k5, k6, k7, k8, k9, k10, k11, k12])

In [6]:
def level(vector):
    '''This function gives the level of the vector k, 
    that is, the number given by sum_i i k_i for a given bosonic 
    state k = (k_1, k_2, ...)'''
    r = len(vector)
    su = 0
    for i in range(r):
        su += (i+1)*vector[i]
    return su

def accel_asc(n):
    '''This is a fast algorithm to generate integer partitions. 
    The author of this beauty is Jerome Kelleher and he argues that it is the 
    fasted algorithm in the market to do this job, see here 
    https://jeromekelleher.net/category/combinatorics.html'''
    a = [0 for i in range(n + 1)]
    k = 1
    y = n - 1
    while k != 0:
        x = a[k - 1] + 1
        k -= 1
        while 2 * x <= y:
            a[k] = x
            y -= x
            k += 1
        l = k + 1
        while x <= y:
            a[k] = x
            a[l] = y
            yield a[:k + 2]
            x += 1
            y -= 1
        a[k] = x + y
        y = x + y - 1
        yield a[:k + 1]

def bosonic_states(lev):
    '''Given a level n, this function gives the 
    vector k (the characteristic class) that belong to this subspace.'''

    vectors_k = [] # Here I create a list to save the tuples
    #my_tuple = () # Here I create a tuple to capture the vectors k 
    if lev < 0 or lev > 12:
        raise ValueError("Level must be greater than zero smaller than 13")
        
    #states = np.zeros(lev)
    #states[0] = lev

    #for my_tuple in product(tuple(range(lev+1)), repeat = lev):
    #    states = np.zeros(lev)
    #    states[0] = lev
    #    if level(my_tuple) == level(states):
            #print(my_tuple)
    for a in accel_asc(lev):
        vec = [0]*lev
        for i in range(len(a)):
            vec[a[i]-1] += 1
            if level(vec) == lev:
                vectors_k.append(vec)

    return vectors_k

def bosonic_states_partition(lev):
    '''Given a level n, this function gives the 
    partitions that belong to this subspace.'''

    partitions = [] # Here I create a list to save the tuples
    if lev < 0 or lev > 12:
        raise ValueError("Level must be greater than zero smaller than 13")

    for a in accel_asc(lev):
        vec = [0]*lev
        for i in range(len(a)):
            vec[i] += a[i]
        vec.sort(reverse=True)
        partitions.append(vec)
            
    return partitions

def mono(vector):
    '''This function gives the monomials in the 
    definition of the complete symmetric polynomials'''
    pro = 1
    for n in range(len(vector)):
        pro *= Rational(1, factorial(vector[n])) * t[n]**(vector[n])
    return pro

def comp_sym_polynomials(lev):
    '''This function gives the complete symmetric polynomials'''
    if lev < 0:
        return 0
    
    elif lev == 0:
        return 1
    else:
        # first of all, given the level above, we need to find the vectors k
        vectors_k = bosonic_states(lev) # This is a list of vectors
        h_poly = 0
 
        for vector in vectors_k:
            h_poly += mono(vector)
        return h_poly

In [42]:
bosonic_states(2)

[[2, 0], [0, 1]]

# Frobenius Character Formula

In [ ]:
# Auxiliary coordinates
x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12 = symbols('x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12')
x = np.array([x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12])

In [ ]:
def VandPol(partition):
    '''Here I want to calculate the Vandermonde polynomia. 
    Try to keep the number of rows of the partition 
    smalled than 12'''
    m = len(partition) # number of rows
    van = 1
    for i in range(m):
        for j in range(m):
            if i < j:
                van *= (x[i] - x[j])
    return van

def NewFactor(j, partition):
    '''In order to calculate the Newton polynomial, we need the factor
    P_j(\vec{x}). The partition gives the number of rows.'''
    m = len(partition) # number of rows
    xj = x**j
    newton_factor = 0
    for j in range(m):
        newton_factor += xj[j]
    return newton_factor
    
def Newton(k, partition):
    '''Calculation of the Newton polynomials themselves.'''
    m = len(partition) # number of rows
    r = len(k)
    Newt = 1
    for j in range(r):
        Newt *= NewFactor(j+1, partition)**(k[j])
    return Newt

def Characters(k, partition):
    l = []
    m = len(partition)
    power = 1 # this is the coefficient in x I need to extract
    for i in range(m):
        l.append(partition[i] + m - i -1) # the minus comes from the fact that python lists start at 0
    for j in range(len(l)):
        power *= x[j]**l[j]
    polynomial = poly(VandPol(partition)*Newton(k, partition))
    coeff = polynomial.coeff_monomial(power)
    return coeff

# Schur Polynomials

In [7]:
def schur(partition):
    '''Here we calculate the Schur polynomial in 
    terms of Miwa coordinates using the determinant formula.'''
    if partition[0] == 0:
        return 1
    else:
        l = len(partition) # length = number of rows in the partition
        H = matrix = zeros(l, l)
        for (n,m) in product(tuple(range(l)), repeat = 2):
            H[n,m] = comp_sym_polynomials( partition[n] - n + m)
        return H.det()

In [8]:
def schurCh(partition):
    '''Here we calculate the Schur polynomial in 
    terms of Miwa coordinates using the characters.'''
    
    lev = sum(partition)

    if lev == 0:
        return 1
    else:
        schur = 0
        vectors = bosonic_states(lev)
        for vector in vectors:
            schur += mono(vector) * Characters(vector, partition)
        return schur

In [10]:
# teste to see if the definitions are equal
part = (4,3,2)
schur(part) - schurCh(part)

0

In [11]:
schur([3,2])

t1**5/24 + t1**3*t2/6 - t1**2*t3/2 + t1*t2**2/2 - t1*t4 + t2*t3

# Skew Schur Polynomials

In [9]:
def skewSchur(partition1, partition2):
    '''Here we calculate the Skew Schur polynomials in 
    terms of Miwa coordinates using the determinant formula.'''
    if len(partition1) != len(partition2):
        raise ValueError("Please, write both partitions in the same form: [l1, ... , lm]")
    elif partition1[0] == 0 and partition2[0] == 0:
        return 1
    else:
        l = len(partition1) # length = number of rows in the partition
        H = matrix = zeros(l, l)
        for (n,m) in product(tuple(range(l)), repeat = 2):
            H[n,m] = comp_sym_polynomials(partition1[n] - partition2[m] - n + m)
        return H.det()

def interlaced(partition1, partition2):
    '''Check if two partitions are interlaced.'''
    n = len(partition1)-1
    count = 0
    for index in range(n):
        if partition1[index] >= partition2[index] and partition2[index] >= partition1[index+1]:
            count += 1
    return n == count

In [13]:
skewSchur([3,2,1], [0,0,0])

t1**6/45 - t1**3*t3/3 + t1*t5 - t3**2

In [35]:
skewSchur([2,1], [0,0]) - schur([2,1])

0

In [105]:
part1 = [2,1,0] 
part2 = [1,0,0]

In [106]:
interlaced(part1, part2)

True